In [1]:
!git clone https://github.com/hein-nkhh/unilm.git
%cd unilm/beit3

Cloning into 'unilm'...
remote: Enumerating objects: 11122, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 11122 (delta 31), reused 18 (delta 18), pack-reused 11079 (from 4)
Receiving objects: 100% (11122/11122), 75.39 MiB | 33.80 MiB/s, done.
Resolving deltas: 100% (5248/5248), done.
/kaggle/working/unilm/beit3


In [2]:
from IPython.display import clear_output

In [3]:
!pip install -r requirements.txt
clear_output()

In [4]:
import os
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer
from modeling_finetune import beit3_large_patch16_384_retrieval
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import autocast
from transformers import XLMRobertaTokenizer
import torch
import json
import cv2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer = XLMRobertaTokenizer("/kaggle/input/beit3_base_retrieval/pytorch/default/2/beit3.spm")

# Mô hình beit_3
ckpt = "/kaggle/input/beit3_base_retrieval/pytorch/default/2/beit3_large_patch16_384_coco_retrieval.pth"
model = beit3_large_patch16_384_retrieval(pretrained=False)
state_dict = torch.load(ckpt, map_location=device)
model.load_state_dict(state_dict["model"], strict=False)
model = model.to(device)
model.eval()
clear_output()

transform  = transforms.Compose([
    transforms.Resize((384, 384), interpolation=3), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

In [6]:
# Mô hình translate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "VietAI/envit5-translation"
tokenizer_translate = AutoTokenizer.from_pretrained(model_name)  
model_translate = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model_translate = model_translate.to('cuda:0')

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

2025-09-04 14:01:46.011545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756994506.333539      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756994506.428279      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

In [7]:
def encode_text(query: str, tokenizer, model, max_len=32, device=device):
    tokens = tokenizer.tokenize(query)
    if len(tokens) > max_len - 2:
        tokens = tokens[:max_len - 2]
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    bos, eos, pad = tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id
    token_ids = [bos] + token_ids + [eos]
    num_tokens = len(token_ids)

    padding_mask = [0] * num_tokens + [1] * (max_len - num_tokens)
    language_tokens = token_ids + [pad] * (max_len - num_tokens)

    language_tokens = torch.tensor(language_tokens, dtype=torch.long).unsqueeze(0).to(device)
    padding_mask = torch.tensor(padding_mask, dtype=torch.long).unsqueeze(0).to(device)

    with torch.no_grad():
        _, language_cls = model(text_description=language_tokens,
                                padding_mask=padding_mask,
                                only_infer=True)
        lang_norm = F.normalize(language_cls, p=2, dim=-1).cpu()  # (1,D)
    return lang_norm

In [8]:
# json_files = ['/kaggle/input/aic-sample-test/keyframes_index/L21_V003_keyframes_index.json',
#              # '/kaggle/input/aic-sample-test/keyframes_index/L21_V006_keyframes_index.json',
#              # '/kaggle/input/aic-sample-test/keyframes_index/L21_V007_keyframes_index.json',
#              # '/kaggle/input/aic-sample-test/keyframes_index/L21_V011_keyframes_index.json'
#             ]


# video_paths = ['/kaggle/input/aic-sample-test/videos/L21_V003.mp4', 
#                # '/kaggle/input/aic-sample-test/videos/L21_V006.mp4',
#                # '/kaggle/input/aic-sample-test/videos/L21_V007.mp4',
#                # '/kaggle/input/aic-sample-test/videos/L21_V011.mp4'
#               ]

# output_dir = 'extracted_frames'
# os.makedirs(output_dir, exist_ok=True)

# image_paths = []

# for i, json_file in enumerate(json_files):
#     with open(json_file, 'r') as f:
#         frame_indices = json.load(f)

#     cap = cv2.VideoCapture(video_paths[i])
    
#     if not cap.isOpened():
#         print(f"Không thể mở video: {video_paths[i]}")
#         continue

#     # tqdm để hiển thị tiến trình
#     for frame_index in tqdm(frame_indices, desc=f"Trích xuất từ video {i+1}/{len(json_files)}"):
#         cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
#         ret, frame = cap.read()
#         if not ret:
#             print(f"⚠️ Không thể đọc frame tại index {frame_index} trong video {i+1}.")
#             continue

#         frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         frame_image = Image.fromarray(frame_rgb)
        
#         frame_path = os.path.join(output_dir, f'video{i+1}_frame_{frame_index}.png')
#         frame_image.save(frame_path)
#         image_paths.append(frame_path)

#     cap.release()
    
# clear_output()
# print(f"Đã trích xuất tổng cộng {len(image_paths)} frame.")
# # print(f"Các đường dẫn frame:", image_paths)

In [9]:
# image_paths = sorted(image_paths)

In [10]:
# embeddings = []
# ids = []

# with torch.no_grad():
#     for img_path in tqdm(image_paths, desc="🔄 Extracting image embeddings"):
#         image = Image.open(img_path).convert("RGB")
#         image_tensor = transform(image).unsqueeze(0).to(device)

#         with autocast(): 
#             vision_cls, _ = model(image=image_tensor, only_infer=True)
#             vision_norm = F.normalize(vision_cls, p=2, dim=-1)

#         embeddings.append(vision_norm.squeeze(0).cpu())   # (D,)
#         ids.append(img_path)

#         del image_tensor, vision_cls, vision_norm
#         torch.cuda.empty_cache()

# image_embeddings = torch.stack(embeddings, dim=0)  # (N,D)
# torch.save({"embeddings": image_embeddings, "ids": ids}, "image_embeddings.pt")
# print("✅ Saved embeddings:", image_embeddings.shape)



In [14]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    force=True,                 # ép ghi đè cấu hình cũ (rất quan trọng trong notebook)
)

logger = logging.getLogger("Embedd Frame")
logger.info("Xin chào")

2025-09-04 14:02:33,139 - Embedd Frame - INFO - Xin chào


In [17]:
def extract_embeddings_batch(image_paths, model, transform, device, batch_size=16):
    """
    Extract embeddings in batches for faster processing
    
    Args:
        image_paths: List of image file paths
        model: Vision model
        transform: Image preprocessing transform
        device: Device to run on
        batch_size: Number of images to process at once
    """
    embeddings = []
    ids = []
    
    # Process images in batches
    with torch.no_grad():
        for i in tqdm(range(0, len(image_paths), batch_size), desc="🔄 Extracting embeddings (batched)"):
            batch_paths = image_paths[i:i + batch_size]
            
            # Load and preprocess batch of images
            batch_images = []
            batch_ids = []
            
            for img_path in batch_paths:
                try:
                    # image = Image.open(img_path).convert("RGB")
                    image_tensor = transform(img_path)
                    batch_images.append(image_tensor)
                    batch_ids.append(img_path)
                except Exception as e:
                    print(f"⚠️ Error loading {img_path}: {e}")
                    continue
            
            if not batch_images:
                continue
                
            # Stack into batch tensor
            batch_tensor = torch.stack(batch_images).to(device)  # (B, C, H, W)
            
            # Extract features for the batch
            with autocast():
                vision_cls_batch = model(image=batch_tensor, only_infer=True)[0]  # (B, D)
                vision_norm_batch = F.normalize(vision_cls_batch, p=2, dim=-1)
            
            # Add to results
            embeddings.extend([emb.cpu() for emb in vision_norm_batch])
            ids.extend(batch_ids)
            
            # Clean up memory
            del batch_tensor, vision_cls_batch, vision_norm_batch
            torch.cuda.empty_cache()
    
    # Stack all embeddings
    image_embeddings = torch.stack(embeddings, dim=0)  # (N, D)
    
    return image_embeddings, ids

In [12]:
def extract_frames_with_opencv(
    video_path: str, 
    target_height: int = 27, 
    target_width: int = 48, 
    target_fps: float = None,        # thêm tuỳ chọn fps
    show_progressbar: bool = False
):
    """
    Extracts frames from a video using OpenCV and returns a list of PIL Images.
    If target_fps is set, frames will be sampled to match that FPS.
    """
    logger.info(f"Opening video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logger.error(f"Failed to open video: {video_path}")
        raise ValueError(f"Failed to open video: {video_path}")

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Nếu có target_fps thì tính step
    if target_fps is not None and target_fps > 0 and video_fps > 0:
        step = int(round(video_fps / target_fps))
        logger.info(f"Video FPS: {video_fps:.2f}, target FPS: {target_fps}, step: {step}")
    else:
        step = 1
        logger.info(f"Video FPS: {video_fps:.2f}, using all frames")

    frames = []

    progress_bar = tqdm(total=total_frames, desc="Extracting frames", unit="frame") if show_progressbar else None

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % step == 0:   # chỉ lấy frame theo step
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_resized = cv2.resize(frame_rgb, (target_width, target_height))
            img_pil = Image.fromarray(frame_resized)
            frames.append(img_pil)

        frame_idx += 1
        if progress_bar:
            progress_bar.update(1)

    cap.release()
    if progress_bar:
        progress_bar.close()
    logger.info(f"Extracted {len(frames)} frames (from {total_frames})")
    return frames

In [15]:
image_paths = extract_frames_with_opencv('/kaggle/input/aic-sample-test/videos/L21_V001.mp4',
                                         show_progressbar=True)

2025-09-04 14:02:36,712 - Embedd Frame - INFO - Opening video: /kaggle/input/aic-sample-test/videos/L21_V001.mp4
2025-09-04 14:02:36,895 - Embedd Frame - INFO - Video FPS: 30.00, using all frames
Extracting frames: 100%|██████████| 37849/37849 [01:30<00:00, 420.50frame/s]
2025-09-04 14:04:06,908 - Embedd Frame - INFO - Extracted 37849 frames (from 37849)


In [ ]:
embeddings, ids = extract_embeddings_batch(
    image_paths=image_paths,
    model=model,
    transform=transform,
    device=device,
    batch_size=64  # Adjust based on your GPU memory
)

🔄 Extracting embeddings (batched):   0%|          | 0/592 [00:00<?, ?it/s]/tmp/ipykernel_36/2964938504.py:41: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
🔄 Extracting embeddings (batched):  18%|█▊        | 106/592 [07:22<34:25,  4.25s/it]

In [ ]:
torch.save({"embeddings": embeddings, "ids": ids}, "image_embeddings.pt")
print("✅ Saved embeddings:", embeddings.shape)

In [ ]:
def translate_vi_en(text: str) -> str:
    # Thêm prefix "vi:" vào input
    prefixed = f"vi: {text}"
    inputs = tokenizer_translate([prefixed], return_tensors="pt", padding=True).input_ids.to('cuda:0')
    outputs = model_translate.generate(inputs, max_length=50)
    translated = tokenizer_translate.decode(outputs[0], skip_special_tokens=True)
    # Xóa prefix "en:" nếu có
    return translated.replace("en:", "").strip()

In [ ]:
text_query = "Băng tan ở Nam Cực"
text_query_en = translate_vi_en(text_query)
lang_vec = encode_text(text_query_en, tokenizer, model)

# Similarity
similarities = torch.matmul(lang_vec, embeddings.t()).squeeze(0)  # (N,)

# Top-K results
topk = torch.topk(similarities, k=10)
print("Top matches:")
for idx, score in zip(topk.indices, topk.values):
    img_path = ids[idx]
    print(img_path, float(score))

    img = Image.open(img_path)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Sim: {score:.4f}")
    plt.show()